# Updated on 2026‑01‑14: cleaned, portfolio ready


**1. Imports and Settings**

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.set_option("display.max_columns", None)

DATA_PATH = Path(
    "C:/Users/AkanbiOlakunle/Documents/Instacart Market Basket Analysis/instacart-market-basket-analysis"
)

In [3]:
import os
os.getcwd()


'c:\\Users\\AkanbiOlakunle\\Documents\\Instacart Market Basket Analysis\\notebooks'

**2. Load Prior Data**

In [4]:
orders = pd.read_csv(DATA_PATH / "orders.csv")
op_prior = pd.read_csv(DATA_PATH / "order_products__prior.csv")
products = pd.read_csv(DATA_PATH / "products.csv")
aisles = pd.read_csv(DATA_PATH / "aisles.csv")
departments = pd.read_csv(DATA_PATH / "departments.csv")

orders_prior = orders[orders["eval_set"] == "prior"]


In [5]:
# Optimize data types to reduce memory footprint
orders["user_id"] = orders["user_id"].astype("int32")
orders["order_number"] = orders["order_number"].astype("int16")
orders["order_id"] = orders["order_id"].astype("int32")
op_prior["order_id"] = op_prior["order_id"].astype("int32")
op_prior["product_id"] = op_prior["product_id"].astype("int16")
op_prior["add_to_cart_order"] = op_prior["add_to_cart_order"].astype("int8")
op_prior["reordered"] = op_prior["reordered"].astype("int8")
products["product_id"] = products["product_id"].astype("int16")
products["aisle_id"] = products["aisle_id"].astype("int8")
products["department_id"] = products["department_id"].astype("int8")

print("✅ Data types optimized for memory efficiency")

✅ Data types optimized for memory efficiency


In [6]:
print(orders.head())
print(orders["eval_set"].unique())


   order_id  user_id eval_set  order_number  order_dow  order_hour_of_day  \
0   2539329        1    prior             1          2                  8   
1   2398795        1    prior             2          3                  7   
2    473747        1    prior             3          3                 12   
3   2254736        1    prior             4          4                  7   
4    431534        1    prior             5          4                 15   

   days_since_prior_order  
0                     NaN  
1                    15.0  
2                    21.0  
3                    29.0  
4                    28.0  
['prior' 'train' 'test']


**3. Merge Prior Orders with Product Metadata**

In [7]:
prior = (op_prior
    .merge(orders_prior, on="order_id", how="left")
    .merge(products, on="product_id", how="left")
    .merge(aisles, on="aisle_id", how="left")
    .merge(departments, on="department_id", how="left")
)

prior.head()


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department
0,2,-32416,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry
3,2,-19618,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry


**4. Build Time Axis (days_since_first_order)**

In [8]:
orders_prior_sorted = orders_prior.sort_values(["user_id", "order_number"])
#orders_prior_sorted.head(10)

In [9]:
orders_prior_sorted["days_since_first_order"] = (
    orders_prior_sorted.groupby("user_id")["days_since_prior_order"].cumsum().fillna(0)
)
#orders_prior_sorted.head()

In [10]:
prior = prior.merge(
    orders_prior_sorted[["order_id", "days_since_first_order"]],
    on="order_id",
    how="left"
)
#prior.head()

In [11]:
prior.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_name,aisle_id,department_id,aisle,department,days_since_first_order
0,2,-32416,1,1,202279,prior,3,5,9,8.0,Organic Egg Whites,86,16,eggs,dairy eggs,28.0
1,2,28985,2,1,202279,prior,3,5,9,8.0,Michigan Organic Kale,83,4,fresh vegetables,produce,28.0
2,2,9327,3,0,202279,prior,3,5,9,8.0,Garlic Powder,104,13,spices seasonings,pantry,28.0
3,2,-19618,4,1,202279,prior,3,5,9,8.0,Coconut Butter,19,13,oils vinegars,pantry,28.0
4,2,30035,5,0,202279,prior,3,5,9,8.0,Natural Sweetener,17,13,baking ingredients,pantry,28.0


**5. Create User-Level Features**

*5.1 Base Aggregates*

In [12]:
user_orders = orders_prior.groupby("user_id").agg(
    n_orders=("order_number", "max"),
    avg_days_between_orders=("days_since_prior_order", "mean")
)


In [13]:
user_orders.head()

,n_orders,avg_days_between_orders
user_id,,
1,10,19.555556
2,14,15.230769
3,12,12.090909
4,5,13.750000
5,4,13.333333


*5.2 Basket Size*

In [14]:
basket_sizes = (prior.groupby(["user_id", "order_id"])["product_id"]
                .count()
                .groupby("user_id")
                .mean()
                .to_frame("avg_basket_size"))


In [15]:
basket_sizes.head()

,avg_basket_size
user_id,
1,5.900000
2,13.928571
3,7.333333
4,3.600000
5,9.250000


*5.3 Reorder Ratio*

In [16]:
user_reorder_ratio = (prior.groupby("user_id")["reordered"]
                      .mean()
                      .to_frame("user_reorder_ratio"))


In [17]:
user_reorder_ratio.head()

,user_reorder_ratio
user_id,
1,0.694915
2,0.476923
3,0.625000
4,0.055556
5,0.378378


*5.4 Combine Features*

In [18]:
user_features = (
    user_orders
    .join(basket_sizes, how="left")
    .join(user_reorder_ratio, how="left")
)
user_features.reset_index(inplace=True)

user_features.head()



,user_id,n_orders,avg_days_between_orders,avg_basket_size,user_reorder_ratio
0,1,10,19.555556,5.900000,0.694915
1,2,14,15.230769,13.928571,0.476923
2,3,12,12.090909,7.333333,0.625000
3,4,5,13.750000,3.600000,0.055556
4,5,4,13.333333,9.250000,0.378378


**6. Create Product-Level Features**

In [19]:
product_features = (
    prior.groupby(["product_id", "product_name"])
    .agg(
        prod_total_purchases=("order_id", "count"),
        prod_n_users=("user_id", "nunique"),
        prod_reorder_rate=("reordered", "mean"),
        prod_avg_add_to_cart=("add_to_cart_order", "mean"),
    )
    .reset_index()
)


product_features.reset_index(inplace=True)

product_features.head()


,index,product_id,product_name,prod_total_purchases,prod_n_users,prod_reorder_rate,prod_avg_add_to_cart
0,0,-32768,"Carrots, Bag",502,294,0.414343,10.358566
1,1,-32767,Organic Biologique Broccoli Florets,7,5,0.285714,5.142857
2,2,-32766,Greek Yogurt Dressing Creamy Ceaser,14,11,0.214286,12.785714
3,3,-32765,"Organic Mango Acai Fruit Leather, 12 Ct",485,150,0.690722,9.828866
4,4,-32764,Shrimp Ceviche,110,56,0.490909,5.572727


**7. User–Product Interaction Features**

In [29]:
up = prior.groupby(["user_id", "product_id"]).agg(
    up_order_count=("order_id", "count"),
    up_first_order_number=("order_number", "min"),
    up_last_order_number=("order_number", "max"),
    up_avg_add_to_cart=("add_to_cart_order", "mean")
)


In [30]:
up.head()

up_order_count  up_first_order_number  \
user_id product_id                                          
1       -29585                   1                     10   
        -26608                   1                     10   
        -25879                   1                     10   
        -23749                   1                      5   
        -19387                   3                      8   

                    up_last_order_number  up_avg_add_to_cart  
user_id product_id                                            
1       -29585                        10                 7.0  
        -26608                        10                 4.0  
        -25879                        10                 3.0  
        -23749                         5                 7.0  
        -19387                        10                 3.0

*7.1 Interaction Order Ratio*

In [31]:
up = up.join(user_features["n_orders"], on="user_id")
up["up_order_ratio"] = up["up_order_count"] / up["n_orders"]


*7.2 Recency Feature Anchored to Last Order*

In [33]:
last_day = (prior.groupby(["user_id", "product_id"])["days_since_first_order"]
            .max()
            .to_frame("up_last_days_since_first_order"))

up = up.join(last_day)


MemoryError: Unable to allocate 247. MiB for an array with shape (32434489,) and data type int64

**8. Save Features**

In [ ]:
# New folder inside current project
features_dir = Path("./my_data/features")
features_dir.mkdir(parents=True, exist_ok=True)
user_features.to_parquet(features_dir / "user_features.parquet", engine="fastparquet")
product_features.to_parquet(
    features_dir / "product_features.parquet", engine="fastparquet"
)
up.reset_index().to_parquet(
    features_dir / "user_product_features.parquet", engine="fastparquet"
)
prior.to_parquet("my_data/features/prior_merged.parquet", engine="fastparquet")


In [ ]:
# Read a Parquet file
ftt = pd.read_parquet("my_data/features/user_features.parquet", engine="fastparquet")
print(ftt.head())

   user_id  n_orders  avg_days_between_orders  avg_basket_size  \
0        1        10                19.555556         5.900000   
1        2        14                15.230769        13.928571   
2        3        12                12.090909         7.333333   
3        4         5                13.750000         3.600000   
4        5         4                13.333333         9.250000   

   user_reorder_ratio  
0            0.694915  
1            0.476923  
2            0.625000  
3            0.055556  
4            0.378378  


In [ ]:
# ---------------------------------------------------
# FINAL INFERENCE FEATURE TABLE (TRAIN / SERVE PARITY)
# ---------------------------------------------------

import gc

model_features = [
    "n_orders_x",
    "avg_days_between_orders",
    "avg_basket_size",
    "user_reorder_ratio",
    "prod_total_purchases",
    "prod_n_users",
    "prod_reorder_rate",
    "prod_avg_add_to_cart",
    "up_order_count",
    "up_first_order_number",
    "up_last_order_number",
    "up_avg_add_to_cart",
    "n_orders_y",
    "up_order_ratio",
    "up_last_days_since_first_order",
]

# Reset index without deep copy + force garbage collection
up_reset = up.reset_index(drop=False)
del up
gc.collect()

# Merge step by step with memory management
print("Merging user features...")
final_features = up_reset.merge(user_features, on="user_id", how="left")
del up_reset
gc.collect()

print("Merging product features...")
final_features = final_features.merge(product_features, on="product_id", how="left")
gc.collect()

# Sanity check
missing = set(model_features) - set(final_features.columns)
assert not missing, f"Missing features: {missing}"

# Select columns directly without copy to avoid memory overhead
# Write directly to parquet with index=False
inference_features = final_features[
    ["user_id", "product_id", "product_name"] + model_features
]

del final_features
gc.collect()

inference_features.to_parquet(
    "./my_data/features/inference_features.parquet", index=False
)

print("✅ inference_features.parquet saved with full model feature set")

NameError: name 'up' is not defined